In [2]:
import os
import time
import datetime
import numpy as np
import PySimpleGUI as sg 

In [3]:
import cv2
import pickle
import matplotlib.pyplot as plt

import pyautogui
from PIL import ImageGrab
from win10toast import ToastNotifier

from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [4]:
from user import User
from kehadiran import Kehadiran

- layout definition

In [5]:
def get_layout_user_item(record):
    Nama = record['Nama']
    NIM  = record['NIM']
    JenisKelamin = record['JenisKelamin']
    JamMasuk = record['JamMasuk']
    filename = record['NamaFoto']
    
    
    layout_photo = [[sg.Image(filename="photo/" + filename, key="foto-" + filename, size=(100,100))]]
    layout_user = [[sg.Text("Nama \t: " + Nama)],
                  [sg.Text("NIM \t: " + NIM)],
                  [sg.Text("Kelamin \t: " + JenisKelamin)],
                  [sg.Text("Masuk \t: " + JamMasuk)]]
    
    layout_item =[
                    sg.Column(layout_photo), 
                    sg.Column(layout_user)
                  ]
    return layout_item

In [6]:
def get_class_info():
    curr_date = datetime.datetime.now().strftime("%A, %d %B %Y")
    layout = [
        [sg.Text(curr_date, justification='right', size=(38,1), font=("Helvetica", 10, "italic"))],
        [sg.Text("Kelas \t\t: "), sg.InputText(key="Kelas", size=(24,1))],
        [sg.Text("Jumlah Peserta \t: "), sg.InputText(key="JumlahPeserta", size=(24,1))],
        [sg.Text("JumlahHadir \t: "), sg.Text("-", key="JumlahHadir", size=(24,1))],
        [sg.Text("JamMulai \t: "), sg.Text("-", key="JamMulai", size=(24,1))],
        [sg.Text("Durasi \t\t: "), sg.Text("-", key="Durasi", size=(24,1))],
        [sg.Text("Status \t\t: "), sg.Text("-", key="Status", size=(24,1))],
        [sg.Button("   Mulai   ", key="mulai"), sg.Button("   Akhiri   ", key="akhiri")]
    ]
    return layout

In [7]:
def get_layout_home(records):
    # list user     
    list_user_layout = [get_layout_user_item(record) for record in records]
    
    layout_left = [
        [
            sg.Column(list_user_layout, scrollable=True, vertical_scroll_only=True, size=(360, 400))
        ]
    ]
    
    
    # informasi kelas     
    layout_right = get_class_info()
    
    layout = [
        [
            sg.Column(layout_left),
            sg.VerticalSeparator(), # vertical separator
            sg.Column(layout_right)
        ]
    ]
    return layout

In [8]:
def get_layout(records):
    layout_home = get_layout_home(records)
    layout_history = [[sg.Text("Histori", size=(80,20))]]
    layout_user = [[sg.Text("User", size=(80,20))]]
    layout_pengaturan = [[sg.Text("Pengaturan", size=(80,20))]]

    layout = [[sg.TabGroup([
                            [sg.Tab('Home', layout_home), 
                             sg.Tab('History', layout_history),
                             sg.Tab('User', layout_user),
                             sg.Tab('Pengaturan', layout_pengaturan)]
                            ])
              ]]
    return layout

- facerecognition

In [9]:
def screen_capture(window_name= 'Zoom Meeting'):
    if window_name not in pyautogui.getAllTitles():
        return None
    try :
        fw = pyautogui.getWindowsWithTitle(window_name)[0]
        fw.maximize()
        pyautogui.click(fw.center)
        fw.activate()
    except :
        pass
    
    time.sleep(1)

    sct = ImageGrab.grab() 
    img = np.array(sct)
    
    return img

In [10]:
def preprocess(img):
    face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    
    img_list = []
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    faces = face_cascade.detectMultiScale(img_gray, 1.3, 5)
    for (x, y, w, h) in faces:
        img_face = img_gray[y:y+h, x:x+w]  # crop face image 
        img_resize = cv2.resize(img_face, (100, 100)) # resize to 100 x 100 pixel
        img_list.append(img_resize)
    return img_list, faces

In [11]:
def read_model(filename, path=""):
    with open(os.path.join(path, filename), 'rb') as in_name:
        model = pickle.load(in_name)
        return model

- main program

In [12]:
def create_window(layout):
    return sg.Window(title="Aplikasi Pencatatan Kehadiran", layout=layout, margins=(10, 10), finalize=True)

In [13]:
def recognizer(labels, window_name= 'Zoom Meeting', show_toast=False, threshold=0.3):
    # get data     
    img = screen_capture(window_name=window_name)
    if img is None :
        print("[INFO] could not find window !")
        return None

    # preprocessing    
    img_list, face_coords = preprocess(img)
    if len(img_list) < 1 :
        print("[INFO] could not find face!")
        return None
    else : 
        # convert each detected face to 1D array feature vector
        img_list_flatten = [img.flatten() for img in img_list]

        # apply PCA to each 1D array feature vector
        img_list_pca = pca.transform(img_list_flatten)

        # predict data using SVM    
        ids = np.array(model_svm.predict(img_list_pca))

        proba = model_svm.predict_proba(img_list_pca)
        confidence = np.array([np.max(p) for p in proba])

        reff = [i for i, conf in enumerate(confidence) if conf >= threshold]

        filtered_ids = ids[reff]
        filtered_confidence = confidence[reff]
        
        label_output = [labels[i] for i in filtered_ids]
        
        for i in range(len(filtered_ids)):
            title = "Name : %s (%.2f%%)" % (label_output[i], (filtered_confidence[i]*100))
            if show_toast:
                toaster.show_toast("Attendance Systems", title)
        
        
    return filtered_ids

In [16]:
toaster = ToastNotifier()

pca = read_model("pca_model.pkl")
model_svm = read_model("SVM_scikit_model.pkl")

kehadiran = Kehadiran()
user = User()
labels = ["-"]*10 
for record in user.select_user():
    labels.insert(record['PredictionId'], record['Nama'])
    
curr_date = datetime.datetime.now().strftime("%Y-%m-%d")

curr_user_records = []
layout = get_layout(curr_user_records)
window = create_window(layout)
window['akhiri'].update(disabled=True)

status = ''
start_time = ''
update = False
delay_capture = 30 #second


while True :
    event, values = window.read(timeout=25)
    
    if event == 'mulai' or update == True:
        window.close()
        
        if event == 'mulai' :
            start_time = datetime.datetime.now()
            
        curr_user_records = user.get_user_in_class(curr_date, values['Kelas'])
        layout = get_layout(curr_user_records)
        window = create_window(layout)
        
        status = 'berlangsung'
        window['Kelas'].update(value=values['Kelas'])
        window['JumlahPeserta'].update(value=values['JumlahPeserta'])
        window['JumlahHadir'].update(value=len(curr_user_records))
        window['Status'].update(value=status)
        window['JamMulai'].update(value=start_time.strftime("%H:%M:%S"))
        
        window['mulai'].update(disabled=True)
        window['akhiri'].update(disabled=False)
        
        if update :
            update = False
        
    if event == 'akhiri':
        status = 'selesai'
        window['Status'].update(value=status)
        window['mulai'].update(disabled=False)
        window['akhiri'].update(disabled=True)
        
        kehadiran.update_kehadiran_selesai(values['Kelas'], 
                                           curr_date, 
                                           datetime.datetime.now().strftime("%Y-%m-%d"))

        
    if status == 'berlangsung':
        curr_time = datetime.datetime.now()
        diff_time = curr_time - start_time
        window['Durasi'].update(value=str(diff_time).split(".")[0])
        
        if curr_time.second % delay_capture == 0 :
            predicted_ids = recognizer(labels, show_toast=False)
            if predicted_ids is not None :
                for idx in predicted_ids :
                    if idx not in [record['PredictionId'] for record in curr_user_records] :
                        record_user = user.select_user_by_prediction_id(int(idx))
                        if record_user is not None :
                            print("Nama :", record_user['Nama'])
                            
                            record_kehadiran = {}
                            record_kehadiran['UserId'] = record_user['Id']
                            record_kehadiran['JamMasuk'] = datetime.datetime.now().strftime("%H:%M:%S")
                            record_kehadiran['NamaKelas'] = values['Kelas']
                            record_kehadiran['JamKelasMulai'] = start_time.strftime("%H:%M:%S")
                            record_kehadiran['JamKelasBerakhir'] = ''
                            record_kehadiran['Status'] = status
                            record_kehadiran['Date'] = curr_date

                            kehadiran.create_kehadiran(record_kehadiran)
                            
                            update = True
                            
            
    if event == sg.WIN_CLOSED:
        break
        
window.close()

Nama : Yunus


In [14]:
kehadiran = Kehadiran()
kehadiran.update_kehadiran_selesai('Sunda', 
                                   datetime.datetime.now().strftime("%Y-%m-%d"), 
                                   datetime.datetime.now().strftime("%H:%M:%S"))

'success'